<a href="https://colab.research.google.com/github/em-chiu/intersection_project/blob/main/EC%20Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [150]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import urllib.request
from nltk.corpus import stopwords
import re
import numpy as np
from collections import Counter
from sklearn import preprocessing
import string
nltk.download('averaged_perceptron_tagger') # to solve pos feature extract issue
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

import pandas as pd
import csv
# import warnings
# warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emilychiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/emilychiu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/emilychiu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [46]:
# load the data into list

# racistfile = open("FTR_new_labels.csv", encoding='latin-1', "r")
# racistdata = list(csv.reader(racistfile, delimiter=","))
# racistfile.close()

In [47]:
# with open('FTR_new_labels.csv', newline='', encoding='latin-1') as csvfile:
#     csv_reader = list(csv.reader(csvfile, delimiter='|'))

#     print(csv_reader)

In [103]:
# import os
# data_dir = '../data/'

# or_fname = 'FTR_new_labels.csv'
# fname = data_dir+or_fname

# load the data to df

df = pd.read_csv('FTR_new_labels.csv', header=None, encoding = "ISO-8859-1")\
        .drop(0, axis=1)\
        .rename(columns={1: 'tweet'})

In [104]:
# look at data
df.head()

,tweet,2
0,tweet,label
1,Elle ne devait pas dÃ©gager la nazi ??? https:...,1
2,je dois faire un oral en anglais sur Hitler et...,0
3,@ambrassmoi_ A Ã§a ð¤ de ressembler Ã la cr...,0
4,"""Par curiositÃ©, jâai voulu Ã©couter les dis...",0


In [105]:
print(type(df))

<class 'pandas.core.frame.DataFrame'>


In [106]:
dfobject=df.applymap(str)
print(dfobject.dtypes)

tweet    object
2        object
dtype: object


In [112]:
dfstring=df.to_string


In [86]:
datalist = df.values.tolist()
# data as list to feed into features functions
print(datalist)


[['tweet', 'label'], ['Elle ne devait pas dÃ©gager la nazi ??? https://t.co/NAGVyrhsXy', '1'], ['je dois faire un oral en anglais sur Hitler et sa propagande pour la section euro Ã§a sâ\x80\x99annonce compliquÃ©e', '0'], ['@ambrassmoi_ A Ã§a ð\x9f¤\x8f de ressembler Ã\xa0 la croix nazi mdrr', '0'], ['"Par curiositÃ©, jâ\x80\x99ai voulu Ã©couter les discours longs de Hitlerâ\x80¦.on en trouve sur internet mais pas sur YouTube.\n', '0'], ['@tical10 @ExBarcaFan74523 @ConflitsFrance Mdr deux nazi n importe quoi ð\x9f¤¦ð\x9f\x8f¼â\x80\x8dâ\x99\x80ï¸\x8f', '1'], ['"@Henry7525 @oooANONYMEooo @yanber_be @alexvizorek La vision sur les choses du passÃ© changent l\'Ã©volution du monde. Mais arrÃªtez de mettre tous les ""wokes"" dans le mÃªme panier en enlevant toute la complexitÃ© et la nuance de la rÃ©alitÃ©. C\'est exactement ce que faisait Hitler avec les juifs."', '0'], ['"Je lis des choses concernant l\'absence de Cyril...\n', '0'], ['"Un docu trÃ¨s trash qui fait littÃ©ralement lâ\x80\x99ap

In [87]:
print(type(datalist))

<class 'list'>


In [139]:
# create a list of corresponding labels
notracist_labels = [0] * 1930 # length of nonracist labels + header
racist_labels = [1] * 927
all_labels = notracist_labels + racist_labels

In [116]:
def tokenize(text):
    tweet = " ".join(re.split("[^a-zA-Z]*", text.lower())).strip()
    return tweet.split(' ')

In [117]:
tokenize(df)

AttributeError: 'DataFrame' object has no attribute 'lower'

In [62]:
def generate_stopwords():
    stopwords_en = nltk.corpus.stopwords.words("english")
    stopwords_fr = nltk.corpus.stopwords.words("french")
    stop_words = stopwords_en+stopwords_fr
    other_exclusions = ["les"]
    #stop_words.extend(other_exclusions)
    return stop_words

def remove_stopwords(text_list, sw):
    return [word for word in text_list if word not in sw]

In [107]:
# extract features: bag of stop words
def remove_stopwords(texts):
    bow = [] 
    stopwords_en = stopwords.words('english')
    stopwords_fr = nltk.corpus.stopwords.words("french")
    stop_words = stopwords_en+stopwords_fr
    other_exclusions = ["les"]
    for text in texts:      
        counts = []
        tokens = nltk.word_tokenize(text)#.lower())
        for sw in stop_words:
            sw_count = tokens.count(sw)
            counts.append(sw_count)
        bow.append(counts)
    bow_np = np.array(bow).astype(float)
    return bow_np

In [113]:
# extract features
stopwords_features = remove_stopwords(dfstring)



TypeError: 'method' object is not iterable

In [ ]:
stop_words_features.shape

(31998, 179)

In [ ]:
# convert features and labels to numpy arrays
X = stop_words_features # count vectorizer
Y = np.array(all_labels)

# run classifier using 10-fold cross validation
# report mean accuracy 

scores = cross_val_score(MultinomialNB(), X, Y, scoring='accuracy', cv=10)
print(scores.mean())

0.8735535323538606


In [ ]:
# extract features: POS tags
def POS_tags(texts):
  bow = [] # bag of word list initialized
  POS = ['NN', 'NNP', 'DT', 'IN', 'JJ', 'NNS','CC','PRP','VB','VBG']
  for text in texts:      
      counts = []
      tokens = nltk.word_tokenize(text.lower()) # tokenized text
      tagged_words = nltk.pos_tag(tokens) # tag tokens, list of tuples
      pos_tags = [x[1] for x in tagged_words] # 2nd element of tuple- POS of the tokens in list
      # print(tokens,'\n', tagged_words, '\n', pos_tags, '\n', POS) # look at variables to differentiate
      for pos in POS:
          pos_count = pos_tags.count(pos)
          counts.append(pos_count)
      bow.append(counts)
  bow_np = np.array(bow).astype(float)
  return bow_np

In [ ]:
# extract features
pos_features = POS_tags(all_headlines)

In [ ]:
pos_features.shape

(31998, 10)

In [ ]:
# convert features and labels to numpy arrays
X = pos_features
Y = np.array(all_labels)

# run classifier using 10-fold cross validation
# report mean accuracy 

scores = cross_val_score(MultinomialNB(), X, Y, scoring='accuracy', cv=10)
print(scores.mean())

0.7672044584245077


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk import ngrams

In [ ]:
# extract features: unigrams
def unigrams_lex(texts):
  unigrams = []
  stop = set(stopwords.words('english'))
  for headline in texts:
    filtered_headlines = []
    tokens = nltk.word_tokenize(headline)
    filtered = [token for token in tokens if not (token in stop or token in string.punctuation)]
    for token in tokens:
        filtered_headlines.append(headline)
        counter_obj = Counter(filtered_headlines)
        top_thirty = counter_obj.most_common(30)
        counts_thirty = [count[1] for count in top_thirty] # takes second element in tuple
    unigrams.append(counts_thirty) # desired: counts for 30 most common unigrams in entire corpus (remove stopwords and punctuation for unigram count)
  unigrams_np = np.array(unigrams, dtype=object)
  return unigrams_np

In [ ]:
unigrams_features = unigrams_lex(all_headlines)

In [ ]:
unigrams_features.shape

(31998, 1)

In [ ]:
# convert features and labels to numpy arrays
X = unigrams_features
Y = np.array(all_labels)

# run classifier using 10-fold cross validation
# report mean accuracy 

scores = cross_val_score(MultinomialNB(), X, Y, scoring='accuracy', cv=10)
print(scores.mean())

0.49996874023132226


In [ ]:
unigram_features = unigrams_lex(all_headlines)

In [ ]:
unigram_features.shape

(31998, 1)

In [ ]:
# extract features: punctuation mark in string.punctuation
def count_puncs(texts):
  bow = []
  punctuation = string.punctuation
  for text in texts:      
      counts = []
      tokens = nltk.word_tokenize(text.lower())
      for punc in punctuation:
          punc_count = tokens.count(punc)
          counts.append(punc_count)
      bow.append(counts)
  bow_np = np.array(bow).astype(float) #converting bow list to numpy array list, converting to float
  return bow_np

In [ ]:
# extract features
punc_features = count_puncs(all_headlines)

In [ ]:
punc_features.shape

(31998, 32)

In [ ]:
# convert features and labels to numpy arrays
X = punc_features
Y = np.array(all_labels)

# run classifier using 10-fold cross validation
# report mean accuracy 

scores = cross_val_score(MultinomialNB(), X, Y, scoring='accuracy', cv=10)
print(scores.mean())

0.5012524812441388


In [ ]:
# change list to string
headlines_str = '\n'.join(all_headlines) # joining and puts on separate lines
#space makes one big line of headlines

In [ ]:
# use tokenized headlines instead of tokenizing each time
tokenized_headlines = nltk.word_tokenize(headlines_str.lower())

In [ ]:
def get_avg_char_per_word(headlines):
  total_length = sum(len(word) for word in headlines) #for word in headline.split())
  num_words = len(tokenized_headlines)
  return total_length/num_words

In [ ]:
def get_uniquetotal_words(headline):
  num_unique_words = len(set(headline)) # one headline at a time to get one unique number at the time of iteration
  total_words = len(headline)
  return num_unique_words/total_words

In [ ]:
def get_num_words(headline):
  return len(headline)

In [ ]:
def get_long_words(headline): # creating new item for list
  for word in headline:
    long_count = 0 # initialize w/ 0 for counting
    if len(word) >= 6:
      long_count += 1 #treating as a variable (answer is number, not list)
  return long_count

In [ ]:
def get_long_words(headline): # creating new item for list
  long_count = 0 # initialize w/ 0 for counting
  for word in headline:
    if len(word) >= 6:
      long_count += 1 #treating as a variable (answer is number, not list)
  return long_count

In [ ]:
get_long_words(all_headlines)

31998

In [ ]:
# extract features: complexity
def complexity(headlines): # passing headlines, list of str (headlines)
  complexity = [] # keeps track of list of headlines
  for headline in headlines:
    headline_features = [] # stores features for array- list affected by for loop, needed to append function values
    tokens = nltk.word_tokenize(headline.lower()) #one headline
    headline_features.append(get_avg_char_per_word(tokens)) #average character per word
    headline_features.append(get_uniquetotal_words(tokens)) # unique/total words
    headline_features.append(len(tokens)) # number of words
    headline_features.append(get_long_words(tokens)) #long words
    complexity.append(headline_features)
  complexity_np = np.array(complexity).astype(float) # array = list of lists    
  return complexity_np

    # looking for 4 complexity features, go through/for every headline, extract 4 features- then save them
#4 subfunctions, 1 for each complexity measure 
#pass a single headline to the function and return a number

In [ ]:
complexity_features = complexity(all_headlines)

In [ ]:
complexity_features.shape

(31998, 4)

In [ ]:
# convert features and labels to numpy arrays
X = complexity_features
Y = np.array(all_labels)

# run classifier using 10-fold cross validation
# report mean accuracy 

scores = cross_val_score(MultinomialNB(), X, Y, scoring='accuracy', cv=10)
print(scores.mean())

0.5893177360112536


In [151]:
def add_lexical_features(dataframe):
    urls = re.compile(r"http")
    ats = re.compile(r"@[a-zA-Z.]*")
    hashtags = re.compile(r"#[a-zA-Z]*")
    letters = re.compile(r"[a-zA-Z]")
    caps = re.compile(r"[A-Z]")
    fancy = [";",'\"','(','<<']

    nbr_characters = [len(s) for s in df.tweet]
    df['nbr_characters'] = pd.DataFrame(nbr_characters, index=df.index)

    nbr_words = [len(s.split()) for s in df.tweet] # to update after cleaning
    df['nbr_words'] = pd.Series(nbr_words, index=df.index)

    nbr_ats = [len(ats.findall(text)) for text in df.tweet]
    df['nbr_ats'] = pd.Series(nbr_ats, index=df.index)

    nbr_hashtags = [len(hashtags.findall(text)) for text in df.tweet]
    df['nbr_hashtags'] = pd.Series(nbr_hashtags, index=df.index)

    nbr_urls = [len(urls.findall(text)) for text in df.tweet]
    df['nbr_urls'] = pd.Series(nbr_urls, index=df.index)

    nbr_letters = [len(letters.findall(text)) for text in df.tweet]
    df['nbr_letters'] = pd.Series(nbr_letters, index=df.index)

    nbr_caps = [len(caps.findall(text)) for text in df.tweet]
    df['nbr_caps'] = pd.Series(nbr_caps, index=df.index)

    nbr_fancy = [sum(1 for c in text if c in fancy) for text in df.tweet]
    df['nbr_fancy'] = pd.Series(nbr_fancy, index=df.index)

    return dataframe

In [152]:
# extract features
lexical_features = add_lexical_features(df)

In [153]:
# check size - 2857 tweets
lexical_features.shape

(2857, 10)

In [154]:
# convert features and labels to numpy arrays
X = lexical_features
Y = np.array(all_labels)

# run classifier using 10-fold cross validation
# report mean accuracy 

scores = cross_val_score(MultinomialNB(), X, Y, scoring='accuracy', cv=10)
print(scores.mean())


accuracy and F1 score

nan


In [131]:
# extract all features
# concatenate all features
a = stop_words_features
b = pos_features
c = unigrams_features
d = punc_features
e = complexity_features
f = crypto_nft_features


NameError: name 'stop_words_features' is not defined

In [ ]:
# args = (a, b, c)
# all_features = np.concatenate(args) # asks dimensions to be the same, 3 args limit

In [ ]:
all_features = np.column_stack((a, b, c, d, e, f))

In [ ]:
# convert features and labels to numpy arrays
X = all_features
Y = np.array(all_labels)

# run classifier using 10-fold cross validation
# report mean accuracy 

scores = cross_val_score(MultinomialNB(), X, Y, scoring='accuracy', cv=10)
print(scores.mean())

0.8893370096123789
